In [1]:
import os
from os.path import join
from adin.settings import BASE_DIR

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

pd.set_option('display.max_rows', 1000)

In [2]:
user=User.objects.all()[0]
user

<User: leon>

In [3]:
led_typ=Ledger_Type.objects.get(abreviation='NI')
led_typ

<Ledger_Type: Nota Interna>

In [4]:
data_df=pd.read_csv(join(BASE_DIR, f"_files/internal_note_data.csv"), keep_default_na=False)
data_df

,fecha_registro,caja,base,retefuente,iva,reteiva,total,concepto,descripcion
0,2022-01-27,Efectivo,150000,0,28500,0,178500,Canon Mensual Arriendo Inmueble^2022-01-01_K1-...,Descuento por dano electrico prolongado.
1,2022-02-07,Efectivo,11667,0,0,0,11667,Canon Mensual Arriendo Inmueble^2022-02-08_A6N...,Ajuste de cargo automatico a valor real.
2,2022-03-09,003-193-745,250000,0,47500,0,297500,Canon Mensual Arriendo Inmueble^2022-01-01_K1-...,Descuento por dano electrico prolongado.
3,2022-03-28,Efectivo,20323,0,0,0,20323,Canon Mensual Arriendo Inmueble^2022-03-28_A6N...,Ajuste de cargo automatico a valor real.
4,2022-09-28,003-193-745,19000,0,0,0,19000,Canon Mensual Arriendo Inmueble^2022-09-01_C18...,Cartera castigada de reteiva descontado de dif...
5,2022-10-10,003-193-745,270,0,0,0,270,Canon Mensual Arriendo Inmueble^2022-10-01_K1-...,Cartera castigada por menor valor de dificil r...


In [5]:
objs_df=data_df.assign(
    accountable_concept=data_df.concepto.apply(lambda x: Accountable_Concept.objects.get(code=x) if Accountable_Concept.objects.filter(code=x) else None),
    holder=data_df.concepto.apply(lambda x: Accountable_Concept.objects.get(pk=x).accountable.ledger_holder()),
    third_party=data_df.concepto.apply(lambda x: Accountable_Concept.objects.get(pk=x).accountable.ledger_third_party()),
    ledger_date=data_df.fecha_registro.apply(lambda x: datetime.date(int(x.split('-')[0]), int(x.split('-')[1]), int(x.split('-')[2])))
)
objs_df

,fecha_registro,caja,base,retefuente,iva,reteiva,total,concepto,descripcion,accountable_concept,holder,third_party,ledger_date
0,2022-01-27,Efectivo,150000,0,28500,0,178500,Canon Mensual Arriendo Inmueble^2022-01-01_K1-...,Descuento por dano electrico prolongado.,Canon Mensual Arriendo Inmueble^2022-01-01_K1-...,"Corkidi Tzuckerman, León","Aguiño Mosquera, Edison",2022-01-27
1,2022-02-07,Efectivo,11667,0,0,0,11667,Canon Mensual Arriendo Inmueble^2022-02-08_A6N...,Ajuste de cargo automatico a valor real.,Canon Mensual Arriendo Inmueble^2022-02-08_A6N...,"Corkidi Yaffe, Isidoro","Pernia Rodriguez, Jonathan",2022-02-07
2,2022-03-09,003-193-745,250000,0,47500,0,297500,Canon Mensual Arriendo Inmueble^2022-01-01_K1-...,Descuento por dano electrico prolongado.,Canon Mensual Arriendo Inmueble^2022-01-01_K1-...,"Corkidi Yaffe, Isidoro","Silva Riaños, Luis Fernando",2022-03-09
3,2022-03-28,Efectivo,20323,0,0,0,20323,Canon Mensual Arriendo Inmueble^2022-03-28_A6N...,Ajuste de cargo automatico a valor real.,Canon Mensual Arriendo Inmueble^2022-03-28_A6N...,"Corkidi Yaffe, Isidoro","Uribe Trujillo, Derly",2022-03-28
4,2022-09-28,003-193-745,19000,0,0,0,19000,Canon Mensual Arriendo Inmueble^2022-09-01_C18...,Cartera castigada de reteiva descontado de dif...,Canon Mensual Arriendo Inmueble^2022-09-01_C18...,"Corkidi Tzuckerman, León",Productos de Caucho y Lona S.A.S.,2022-09-28
5,2022-10-10,003-193-745,270,0,0,0,270,Canon Mensual Arriendo Inmueble^2022-10-01_K1-...,Cartera castigada por menor valor de dificil r...,Canon Mensual Arriendo Inmueble^2022-10-01_K1-...,"Corkidi Tzuckerman, León",Representaciones Especiales LTDA.,2022-10-10


In [6]:
def create_receipt(data):
    cha_data = {}
    if data.base > 0:
        cha_data[13802005] = -data.base
        cha_data[913005] = data.base
    if data.iva > 0:
        cha_data[13050505] = -data.iva
        cha_data[2408] = data.iva
    ledger=Ledger(
        state_change_user=user,
        type=led_typ,
        holder=data.holder,
        third_party=data.third_party,
        date=data.ledger_date,
        description=data.descripcion
    )
    ledger.save()
    for acc, value in cha_data.items():
        Charge(
            state_change_user=user,
            ledger=ledger,
            account=Account.objects.get(code=acc),
            value=value,
            concept=data.accountable_concept
        ).save()
        
    
    

In [7]:
objs_df.apply(lambda x: create_receipt(x), axis=1)

0    None
1    None
2    None
3    None
4    None
5    None
dtype: object